1
2
3
4
5
6
Libraries imported.


<h2>The purpose of this notebook is to build the final assignment of the IBM courses for Data Science :<h2/>
<h1>The capstone project<h1/>

<h3>Introduction/Business Problem<h3/>

I imagine working with a company offering business analysis to others. Our current client, Paul, is a person whose wish is to create his own Chinese/Mexican restaurant in Toronto. Yes it's quiet odd...  He does not have a particular preference about the location of his future restaurant, but would like to have it placed in a strategic position, without any cost consideration at first. That is why he asked us to help him : finding the neighborhood in Toronto which is the most interesting considering both:
Presence of potential clients
Competition with similar restaurants

<h3>Data<h3/>

To deal with this problem, I will be using the Foursquare location data. The idea is to make requests to get the venues in each of Toronto's neighborhoods. Then we will be able to work with them. In order to locate the neighborhoods and make Foursquare requests, I will first use the Wikipedia page listing these neighborhoods per postal codes. 

<h3>Methodology<h3/>

In [120]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis


print('Libraries imported.')

Libraries imported.


<h3>Wikipedia scraping<h3/>

In [121]:
link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
import urllib.request

page = urllib.request.urlopen(link)
type(page)

from bs4 import BeautifulSoup
soup = BeautifulSoup(page, "lxml")

right_table=soup.find('table', class_='wikitable sortable')

A=[]
B=[]
C=[]
for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if (len(cells)==3):
        A.append(cells[0].find(text=True)[0:-1])
        B.append(cells[1].find(text=True)[0:-1])
        C.append(cells[2].find(text=True)[0:-1])

df=pd.DataFrame(A,columns=['Postal Code'])
df['Borough']=B
df['Neighbourhood']=C


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


<h3>Pandas processing<h3/>

In [123]:
df = df[df.Borough != 'Not assigned']
df.reset_index(drop=True, inplace=True)
df_coord = pd.read_csv('http://cocl.us/Geospatial_data')
toronto_df = pd.merge(df, df_coord, on = ['Postal Code'])
toronto_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


<h3>Mapping of the bouroughs<h3/>

In [54]:
!pip install folium

     |████████████████████████████████| 102kB 7.9MB/s ta 0:00:011


In [60]:
import folium
import requests
from pandas.io.json import json_normalize

In [61]:
latitude = toronto_df['Latitude'].mean(axis=0)
longitude = toronto_df['Longitude'].mean(axis=0)

In [62]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, borough, neighbourhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<h3>Extracting and exploring our dataset<h3/>

In [67]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [68]:
toronto_venues = getNearbyVenues(names=toronto_df['Neighbourhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [69]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 239 uniques categories.


<h3>Counting the restaurants, Chinese and Mexican restaurants in each borough<h3/>

In [80]:
restaurant_df = toronto_venues[toronto_venues['Venue Category'].str.find('Restaurant')>-1]
chinese_df = toronto_venues[toronto_venues['Venue Category'].str.find('Chinese')>-1]
mexican_df = toronto_venues[toronto_venues['Venue Category'].str.find('Mexican')>-1]
restaurant_df.reset_index(inplace=True)
chinese_df.reset_index(inplace=True)
mexican_df.reset_index(inplace=True)
print(restaurant_df.shape)
print(chinese_df.shape)
print(mexican_df.shape)
restaurant_df.head()

(301, 8)
(10, 8)
(12, 8)


,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
1,11,"Regent Park, Harbourfront",43.654260,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
2,29,"Regent Park, Harbourfront",43.654260,-79.360636,El Catrin,43.650601,-79.358920,Mexican Restaurant
3,30,"Regent Park, Harbourfront",43.654260,-79.360636,Cluny Bistro & Boulangerie,43.650565,-79.357843,French Restaurant
4,39,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,Lac Vien Vietnamese Restaurant,43.721259,-79.468472,Vietnamese Restaurant


<h3>Counting the restaurants and sushi restaurants in each borough<h3/>

In [71]:
restaurants_count = restaurant_df[['Neighborhood', 'Venue']].groupby('Neighborhood').count().reset_index()
restaurants_count.rename(columns={"Venue": "Restaurants Count"}, inplace=True)
restaurants_count.set_index('Neighborhood', inplace=True)
restaurants_count.head()

,Restaurants Count
Neighborhood,
Agincourt,1
"Bathurst Manor, Wilson Heights, Downsview North",3
Bayview Village,2
"Bedford Park, Lawrence Manor East",9
Berczy Park,6


In [73]:
chinese_count = chinese_df[['Neighborhood', 'Venue']].groupby('Neighborhood').count().reset_index()
chinese_count.rename(columns={"Venue": "Chinese Restaurants Count"}, inplace=True)
chinese_count.set_index('Neighborhood', inplace=True)
chinese_count.head()

,Chinese Restaurants Count
Neighborhood,
Bayview Village,1
"Clarks Corners, Tam O'Shanter, Sullivan",1
Don Mills,1
"Dorset Park, Wexford Heights, Scarborough Town Centre",1
"Kennedy Park, Ionview, East Birchmount Park",1


In [82]:
mexican_count = mexican_df[['Neighborhood', 'Venue']].groupby('Neighborhood').count().reset_index()
mexican_count.rename(columns={"Venue": "Mexican Restaurants Count"}, inplace=True)
mexican_count.set_index('Neighborhood', inplace=True)
mexican_count.head()

,Mexican Restaurants Count
Neighborhood,
Church and Wellesley,1
"Garden District, Ryerson",1
"Guildwood, Morningside, West Hill",1
"High Park, The Junction South",2
"Kensington Market, Chinatown, Grange Park",2


In [102]:
final_df = restaurants_count.join(chinese_count).join(mexican_count)

final_df = final_df.join(toronto_df.set_index('Neighbourhood'))

final_df.reset_index(inplace=True)

final_df.rename(columns={"index":"Neighborhood"}, inplace=True)
final_df.replace(np.nan, 0.0, inplace = True)
final_df.head()

,Neighborhood,Restaurants Count,Chinese Restaurants Count,Mexican Restaurants Count,Postal Code,Borough,Latitude,Longitude
0,Agincourt,1,0.0,0.0,M1S,Scarborough,43.794200,-79.262029
1,"Bathurst Manor, Wilson Heights, Downsview North",3,0.0,0.0,M3H,North York,43.754328,-79.442259
2,Bayview Village,2,1.0,0.0,M2K,North York,43.786947,-79.385975
3,"Bedford Park, Lawrence Manor East",9,0.0,0.0,M5M,North York,43.733283,-79.419750
4,Berczy Park,6,0.0,0.0,M5E,Downtown Toronto,43.644771,-79.373306


<h3>Final scoring and mapping<h3/>

In [111]:
final_df['Score'] = final_df['Restaurants Count']/(1+final_df['Chinese Restaurants Count']+final_df['Mexican Restaurants Count'])
final_df['Score'] = round(final_df['Score']/final_df['Score'].max()*20, 1)
final_df.sort_values(by='Score', ascending=False, inplace=True)

In [112]:
final_df.head()

,Neighborhood,Restaurants Count,Chinese Restaurants Count,Mexican Restaurants Count,Postal Code,Borough,Latitude,Longitude,Score
35,"Little Portugal, Trinity",14,0.0,0.0,M6J,West Toronto,43.647927,-79.419750,20.0
56,"The Danforth West, Riverdale",13,0.0,0.0,M4K,East Toronto,43.679557,-79.352188,18.6
63,"Willowdale, Willowdale East",10,0.0,0.0,M2N,North York,43.770120,-79.408493,14.3
14,"Commerce Court, Victoria Hotel",10,0.0,0.0,M5L,Downtown Toronto,43.648198,-79.379817,14.3
22,"First Canadian Place, Underground city",10,0.0,0.0,M5X,Downtown Toronto,43.648429,-79.382280,14.3


In [115]:
mapping_df = final_df.head(15)
mapping_df

latitude = mapping_df['Latitude'].mean(axis=0)
longitude = mapping_df['Longitude'].mean(axis=0)

map_restaurant = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, score, neighborhood in zip(mapping_df['Latitude'], mapping_df['Longitude'], mapping_df['Score'], mapping_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, score)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_restaurant)  
    
map_restaurant

<h3>Results<h3/>

Thanks to our analysis we narrowed all the possible locations for a new Chineses/Mexican restaurant to a few. The score attibuted to every neighborhood shows that 2 of them seem over the others, with a score over 18.6 or 20. These places are indeed rich in restaurants, and thus in clients, but do not have many Chinese restaurants yet, nether Mexican. If our client is not interested by these 2 locations for any reason (financial for instance), we are able to give him an ordered list of the most interesting neighborhoods to install his restaurant.

<h3>Discussion<h3/>

As said earlier, there are 3 neighborhoods that seem very interseting for our client. Namely :

Little Portugal, Trinity	

The Danforth West, Riverdale

Willowdale, Willowdale East	

These location aren't in the center of Toronto, we can guess that the renting price is low. 

<h3>Conclusion<h3/>

We were able to find some locations that seem to be a good compromise between effectiveness and cost. We truely hope that our analysis will help Paul to find the perfect location for his restaurant.